In [1]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '../data/IEEEfraud/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = 'IEEEfraudModels/'  # where to store trained models

train_identity = pd.read_csv(directory+'train_identity.csv.zip')
train_transaction = pd.read_csv(directory+'train_transaction.csv.zip')

In [2]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [3]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='best_quality', time_limit=3600
)

results = predictor.fit_summary()

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'verbosity': 3}
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "IEEEfraudModels/"
AutoGluon Version:  0.2.0
Train Data Rows:    590540
Train Data Columns: 433
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' 

Saving IEEEfraudModels/utils/data/y.pkl
Model configs that will be trained (in order):
	KNeighborsUnif_BAG_L1: 	{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif', 'model_type': <class 'autogluon.tabular.models.knn.knn_model.KNNModel'>, 'priority': 100}, 'ag_args_ensemble': {'use_child_oof': True}}
	KNeighborsDist_BAG_L1: 	{'weights': 'distance', 'ag_args': {'name_suffix': 'Dist', 'model_type': <class 'autogluon.tabular.models.knn.knn_model.KNNModel'>, 'priority': 100}, 'ag_args_ensemble': {'use_child_oof': True}}
	LightGBMXT_BAG_L1: 	{'extra_trees': True, 'ag_args': {'name_suffix': 'XT', 'model_type': <class 'autogluon.tabular.models.lgb.lgb_model.LGBModel'>, 'priority': 90}, 'ag_args_ensemble': {}}
	LightGBM_BAG_L1: 	{'ag_args': {'model_type': <class 'autogluon.tabular.models.lgb.lgb_model.LGBModel'>, 'priority': 90}, 'ag_args_ensemble': {}}
	RandomForestGini_BAG_L1: 	{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass'], 'model_type

limit_output extension: Maximum message size of 10000 exceeded with 10025 characters

In [11]:
ids = ['id-01', 'id-02', 'id-03', 'id-04', 'id-05', 'id-06', 'id-07', 'id-08', 'id-09', 'id-10', 'id-11', 'id-12', 'id-13', 'id-14',
       'id-15', 'id-16', 'id-17', 'id-18', 'id-19', 'id-20', 'id-21', 'id-22',
       'id-23', 'id-24', 'id-25', 'id-26', 'id-27', 'id-28', 'id-29', 'id-30',
       'id-31', 'id-32', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38']
ids = ['_'.join(id.split('-')) for id in ids]
ids

['id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_32',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38']

In [12]:
test_identity = pd.read_csv(directory+'test_identity.csv.zip')
test_identity.columns = ['TransactionID'] + ids + ['DeviceType', 'DeviceInfo']

test_transaction = pd.read_csv(directory+'test_transaction.csv.zip')
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

y_predproba = predictor.predict_proba(test_data)
y_predproba.head(5)  # some example predicted fraud-probabilities

Loading: IEEEfraudModels/models/WeightedEnsemble_L3/model.pkl
Loading: IEEEfraudModels/models/LightGBMXT_BAG_L1/model.pkl
Loading: IEEEfraudModels/models/LightGBMXT_BAG_L2/model.pkl


,0,1
0,0.998239,0.001761
1,0.999453,0.000547
2,0.999484,0.000516
3,0.998583,0.001417
4,0.999254,0.000746


In [13]:
predictor.positive_class

1

In [14]:
predictor.class_labels  # classes in this list correspond to columns of predict_proba() output

[0, 1]

In [15]:
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)

Loading: IEEEfraudModels/models/WeightedEnsemble_L3/model.pkl
Loading: IEEEfraudModels/models/LightGBMXT_BAG_L1/model.pkl
Loading: IEEEfraudModels/models/LightGBMXT_BAG_L2/model.pkl


In [16]:
submission = pd.read_csv(directory+'sample_submission.csv.zip')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv('my_submission.csv', index=False)